# Serialize and Quantize GPT2
Taken ref from: https://www.kaggle.com/code/nulldata/fine-tuning-gpt-2-to-generate-netlfix-descriptions/notebook

Which took ref from: https://medium.com/geekculture/fine-tune-eleutherai-gpt-neo-to-generate-netflix-movie-descriptions-in-only-47-lines-of-code-40c9b4c32475

## Setup

In [1]:
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [2]:
%%capture
if IN_COLAB:
    
    #Remove not needed python versions to free space
    !rm -rf "/usr/local/lib/python2.7"
    !rm -rf "/usr/lib/python2.7"

    # Clone the repo.
    # git clone ""

    #Get any necessary accessory files
    # !wget "https://raw.githubusercontent.com/microsoft/onnxruntime/main/onnxruntime/python/tools/transformers/models/gpt2/convert_to_onnx.py" 

    # Change the working directory to the repo root.
    # %cd

    # Add the repo root to the Python path.
    # import sys, os
    # sys.path.append(os.getcwd())
    
    #Install packages not native to colab
    !pip install python-dotenv
    !pip install transformers
    !pip install transformers[onnx]
    !pip install optimum --upgrade
    !pip install optimum[onnxruntime] --upgrade
    !pip install datasets
    !pip install wandb --upgrade
    # !pip install pandas-profiling --upgrade

    #Mount GDrive to access .env file
    from google.colab import drive
    drive.mount('/content/gdrive')

    #Load env file
    #NOTE: gdrive wont allow you to mount dotfiles
    from dotenv import load_dotenv
    load_dotenv("./gdrive/MyDrive/my_env_file")

In [3]:
import wandb
wandb.login()

ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: a-sh0ts. Use `wandb login --relogin` to force relogin


True

In [4]:
_model_conf = {
    
}

In [5]:
project_name = "gpt2-netflix"
run_name = "quantize_and_serialize_gpt2_onnx"
run_type = "productionize"

In [6]:
run = wandb.init(
        project=project_name, job_type=run_type, name=run_name, config = _model_conf)

In [7]:
model_checkpoint = run.use_artifact("gpt2-netflix-hf:latest").download()

wandb: Downloading large artifact gpt2-netflix-hf:latest, 1379.04MB. 7 files... Done. 0:0:19.5


In [8]:
import os
from optimum.onnxruntime import ORTModelForCausalLM
from transformers import AutoTokenizer

model_save_directory = os.path.join("onnx", "original")
model_file_name = "model.onnx"
common_save_directory = os.path.join("onnx", "common")
tokenizer_save_directory = os.path.join(common_save_directory, "tokenizer")

# Load a model from transformers and export it through the ONNX format
model = ORTModelForCausalLM.from_pretrained(model_checkpoint, from_transformers=True)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

# Save the onnx model and tokenizer
model.save_pretrained(model_save_directory, file_name=model_file_name)
tokenizer.save_pretrained(tokenizer_save_directory)

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/models/gpt2/modeling_gpt2.py:813: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if batch_size <= 0:
/usr/local/lib/python3.7/dist-packages/transformers/models/gpt2/modeling_gpt2.py:198: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  value.size(-1) ** 0.5, dtype=attn_weights.dtype, device=attn_weights.device
/usr/local/lib/python3.7/dist-packages/transformers/models/gpt2/modeling_gpt2.py:198: UserWarning: To copy construct from a tensor, it is recommended to use sou

('onnx/common/tokenizer/tokenizer_config.json',
 'onnx/common/tokenizer/special_tokens_map.json',
 'onnx/common/tokenizer/vocab.json',
 'onnx/common/tokenizer/merges.txt',
 'onnx/common/tokenizer/added_tokens.json',
 'onnx/common/tokenizer/tokenizer.json')

In [9]:
from optimum.onnxruntime.configuration import AutoQuantizationConfig
from optimum.onnxruntime import ORTQuantizer

quantized_save_directory = os.path.join("onnx", "quantized")

# Define the quantization methodology
qconfig = AutoQuantizationConfig.arm64(is_static=False, per_channel=False)
quantizer = ORTQuantizer.from_pretrained(model)

# Apply dynamic quantization on the model
quantizer.quantize(
    save_dir=quantized_save_directory,
    quantization_config=qconfig,
)

PosixPath('onnx/quantized')

Save Model to GCS

In [10]:
#workaround for seldon

In [11]:
from google.colab import auth
auth.authenticate_user()

# https://cloud.google.com/resource-manager/docs/creating-managing-projects
project_id = 'wandb-growth'
!gcloud config set project {project_id}

Updated property [core/project].


In [12]:
import uuid

# Make a unique bucket to which we'll upload the file.
# (GCS buckets are part of a single global namespace.)
bucket_name = 'trained-gpt2-netflix-' + str(uuid.uuid1())

# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/mb
!gsutil mb gs://{bucket_name}

Creating gs://trained-gpt2-netflix-d91f9f82-3561-11ed-9e10-0242ac1c0002/...


In [13]:
# Copy the file to our new bucket.
# Full reference: https://cloud.google.com/storage/docs/gsutil/commands/cp
!gsutil cp -r onnx gs://{bucket_name}/

Copying file://onnx/quantized/ort_config.json [Content-Type=application/json]...
Copying file://onnx/quantized/model_quantized.onnx [Content-Type=application/octet-stream]...
==> NOTE: You are uploading one or more large file(s), which would run
significantly faster if you enable parallel composite uploads. This
feature can be enabled by editing the
"parallel_composite_upload_threshold" value in your .boto
configuration file. However, note that if you do this large files will
be uploaded as `composite objects
<https://cloud.google.com/storage/docs/composite-objects>`_,which
means that any user who downloads such objects will need to have a
compiled crcmod installed (see "gsutil help crcmod"). This is because
without a compiled crcmod, computing checksums on composite objects is
so slow that gsutil disables downloads of composite objects.

Copying file://onnx/common/tokenizer/tokenizer_config.json [Content-Type=application/json]...
Copying file://onnx/common/tokenizer/vocab.json [Conten

Save reference to artifact and move to model registry

Tokenizer

In [14]:
tokenizer_artifact_name = "gpt2-netflix-tokenizer"
tokenizer_artifact_type = "model-assets"
tokenizer_artifact_description = "GPT2 tokenizer for model finetuned on Netflix data"

In [15]:
tokenizer_model_artifact = wandb.Artifact(name=tokenizer_artifact_name, type=tokenizer_artifact_type, description=tokenizer_artifact_description)

In [16]:
tokenizer_model_artifact.add_dir("onnx/common/tokenizer")

wandb: Adding directory to artifact (./onnx/common/tokenizer)... Done. 0.1s


In [17]:
run.log_artifact(tokenizer_model_artifact)

Orginal onnx

In [18]:
onnx_artifact_name = "gpt2-netflix-onnx"
onnx_artifact_type = "model"
onnx_artifact_description = "GPT2 model finetuned as per this article: https://www.kaggle.com/code/nulldata/fine-tuning-gpt-2-to-generate-netlfix-descriptions/notebook exported to ONNX"

In [19]:
onnx_model_artifact = wandb.Artifact(name=onnx_artifact_name, type=onnx_artifact_type, description=onnx_artifact_description)

In [20]:
#workaround for seldon
onnx_model_artifact.add_reference(f"gs://{bucket_name}/onnx/original")

wandb: Generating checksum for up to 10000 objects with prefix "onnx/original"... Done. 0.1s


[<ManifestEntry ref: gs://trained-gpt2-netflix-d91f9f82-3561-11ed-9e10-0242ac1c0002/onnx/original/config.json/config.json>,
 <ManifestEntry ref: gs://trained-gpt2-netflix-d91f9f82-3561-11ed-9e10-0242ac1c0002/onnx/original/model.onnx/model.onnx>]

In [21]:
onnx_model_artifact.metadata.update({"modelURI": f"gs://{bucket_name}/onnx/original/model.onnx"})

In [22]:
run.log_artifact(onnx_model_artifact)

Quantized onnx

In [23]:
quantized_onnx_artifact_name = "gpt2-netflix-onnx-quantized"
quantized_onnx_artifact_type = "model"
quantized_onnx_artifact_description = "GPT2 model finetuned as per this article: https://www.kaggle.com/code/nulldata/fine-tuning-gpt-2-to-generate-netlfix-descriptions/notebook exported to ONNX and quantized"

In [24]:
quantized_onnx_model_artifact = wandb.Artifact(name=quantized_onnx_artifact_name, type=quantized_onnx_artifact_type, description=quantized_onnx_artifact_description)

In [25]:
#workaround for seldon
quantized_onnx_model_artifact.add_reference(f"gs://{bucket_name}/onnx/quantized")

wandb: Generating checksum for up to 10000 objects with prefix "onnx/quantized"... Done. 0.1s


[<ManifestEntry ref: gs://trained-gpt2-netflix-d91f9f82-3561-11ed-9e10-0242ac1c0002/onnx/quantized/model_quantized.onnx/model_quantized.onnx>,
 <ManifestEntry ref: gs://trained-gpt2-netflix-d91f9f82-3561-11ed-9e10-0242ac1c0002/onnx/quantized/ort_config.json/ort_config.json>]

In [26]:
quantized_onnx_model_artifact.metadata.update({"modelURI": f"gs://{bucket_name}/onnx/quantized/model_quantized.onnx"})

In [27]:
run.log_artifact(quantized_onnx_model_artifact)

In [28]:
run.finish()

Promote model to registry
We will be forcing the quantized one to be the selected registered model

In [29]:
wandb_entity = "a-sh0ts"
collection_name = "generative-netflix"

In [30]:
retrieved_model_art = wandb.Api().artifact(f"{wandb_entity}/{project_name}/{quantized_onnx_artifact_name}:latest")

In [33]:
retrieved_model_art.link(f"{wandb_entity}/{project_name}/{collection_name}")

True